In [ ]:
# GENERATE FILE FOR probeTm.py WITH TWO COLUMNS: SEQUENCE NUBMER AND PROBE SEQUENCE #
read_filename = 'data/mouse_chr6/rosa26_probes_sC.bed'
write_filename = 'data/mouse_chr6/rosa26_probes_TM.txt'

# read file output from structureCheck.py #
with open(read_filename) as file:
    probes = [(k+1, line.split('	')[3]) for k, line in enumerate(file)]

# write to new file for input to probeTm.py #
with open(write_filename, 'w') as file:
    for probe in probes:
        # file.write(f'{probe[0]}\t{probe[1]}\n')
        pass

In [1]:
# PLOTTING RESULTS FROM duplex_prob.py #
from duplex_prob import plot_duplex_prob

filtered_filename = 'data/target_seqs/rosa26_against_full/rosa26_probes_sC_filtered.txt'

###################################################################################################

plot_duplex_prob(filtered_filename)

['1', 'GGGAGGAAAAGAAGCCAATCAGCGACGAGACGTCGG', '[0.81839899 0.65538112 0.33560065 0.08462265 0.02709411 0.00910175]']
['2', 'CAAGTGCTTCCTGCCTCCAAAACAGCCCAACTGAGC', '[0.8074672  0.6201455  0.27239797 0.05632009 0.0170688  0.00465417]']
['3', 'CCCTTACTGATAACAACTCAGAGCGACTTTGGGAGAG', '[0.8074672  0.6201455  0.27239797 0.05632009 0.0170688  0.00465417]']
['4', 'GAACTCCCAGAAAGGTATTGCAACACTCCCCTCCCC', '[0.81543772 0.60521875 0.21347395 0.03128633 0.00769808 0.00113579]']
['5', 'TTTAAGCCTGCCCAGAAGACTCCCGCCCATCTTCTA', '[0.79604135 0.5835962  0.21720679 0.03709984 0.01071219 0.0023747 ]']
['6', 'TTTACACCTGTTCAATTCCCCTGCAGGACAACGCCCAC', '[8.69630403e-01 6.85707074e-01 2.39727088e-01 2.68779890e-02']
['5.25931514e-03 4.99669417e-04]']
['7', 'AAACCGAAAATCTGTGGGAAGTCTTGTCCCTCCAAT', '[8.69630403e-01 6.85707074e-01 2.39727088e-01 2.68779890e-02']
['5.25931514e-03 4.99669417e-04]']
['8', 'AGATGACTACCTATCCTCCCATTTTCCTTATTTGCCCCTAT', '[0.89219888 0.7752089  0.43644797 0.09309665 0.02134141 0.00377519]']


NameError: name 'all_probs' is not defined

<Figure size 800x600 with 0 Axes>

In [1]:
from duplex_prob import filter_duplex_prob

sam_file = 'data/target_seqs/rosa26_against_full/rosa26.sam'
bed_file = 'data/target_seqs/rosa26_against_full/rosa26_probes_sC.bed'

filter_duplex_prob(sam_file, bed_file, filter_temp=42, filter_prob=0.2)

<class 'list'>


TypeError: unsupported format string passed to numpy.ndarray.__format__